In [13]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV

In [2]:

df = pd.read_csv("exoplanet_data.csv")
df = df.dropna(axis='columns', how='all')
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Create a Train Test Split

Use `koi_disposition` for the y values

In [3]:
X = df.drop('koi_disposition', axis = 1)
X.head()
y = df['koi_disposition']


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = 0.80)

#A second split for a Hyperparameter tuning set
X_train, X_hp_train, y_train, y_hp_train = train_test_split(X_train, y_train, random_state=0, train_size = 0.5)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [4]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
svc_minmax_scaler = MinMaxScaler().fit(X_train)

X_train_minmax = svc_minmax_scaler.transform(X_train)
X_hp_train_minmax = svc_minmax_scaler.transform(X_hp_train)
X_test_minmax = svc_minmax_scaler.transform(X_test)


# Train the Model



In [5]:

svc_model = SVC()
svc_model = svc_model.fit(X_train_minmax,y_train)

print(f"Training Data Score: {svc_model.score(X_train_minmax, y_train)}")
print(f"Testing Data Score: {svc_model.score(X_test_minmax, y_test)}")

Training Data Score: 0.829041487839771
Testing Data Score: 0.8447782546494993


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [6]:
# Create the GridSearchCV model
### ESP - High cv because scores are very close, and I want to make sure I get a representative average

param_grid = {
            'C': [25, 50, 75],
            'gamma': [0.00005, 0.0001, 0.0005],
            'degree': [1,2,3],
            'tol': [0.01, 0.005, 0.001],
            'kernel': ['linear', 'poly', 'rbf']
            }
grid = GridSearchCV(svc_model, param_grid, verbose=3, cv = 7, n_jobs = -3)

In [7]:
# Train the model with GridSearch
grid.fit(X_hp_train_minmax, y_hp_train)

Fitting 10 folds for each of 1280 candidates, totalling 12800 fits
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-3)]: Done  20 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-3)]: Done 116 tasks      | elapsed:   35.4s
[Parallel(n_jobs=-3)]: Done 276 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-3)]: Done 500 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-3)]: Done 788 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-3)]: Done 1140 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-3)]: Done 1556 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-3)]: Done 2036 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-3)]: Done 2580 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-3)]: Done 3188 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-3)]: Done 3860 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-3)]: Done 4596 tasks      | elapsed: 20.2min
[Parallel(n_jobs=-3)]: Done 5396 tasks      | elapsed: 23.4min
[Parallel(n_jobs=-3)]: Done 6260 tasks     

GridSearchCV(cv=10, estimator=SVC(), n_jobs=-3,
             param_grid={'C': [1, 5, 10, 50], 'degree': [2, 3, 4, 5],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                         'tol': [0.005, 0.001, 0.0001, 1e-05, 1e-06]},
             verbose=3)

In [8]:
print(grid.best_params_)
print(grid.best_score_)


{'C': 50, 'degree': 2, 'gamma': 0.0001, 'kernel': 'linear', 'tol': 0.005}
0.8805542754736303


In [12]:
svc_model = SVC(C= 50, degree= 2, gamma= 0.0001, kernel= 'linear', tol= 0.005)
svc_model = svc_model.fit(X_train_minmax,y_train)

print(f"Training Data Score: {svc_model.score(X_train_minmax, y_train)}")
print(f"Testing Data Score: {svc_model.score(X_test_minmax, y_test)}")

Training Data Score: 0.8837625178826896
Testing Data Score: 0.8934191702432046


In [14]:
with open('pickle_jar/svc_model.pickle', 'wb') as dill:
    pickle.dump(svc_model,dill)
with open('pickle_jar/svc_minmax_scaler.pickle', 'wb') as gherkin:
    pickle.dump(svc_minmax_scaler,gherkin)